In [3]:
import cv2
import numpy as np
import os
import random

ModuleNotFoundError: No module named 'cv2'

In [4]:
a = os.listdir('captcha')

In [64]:
def image_closing(dir,num):
    img = cv2.imread(dir,0) 
    kernel = np.ones((3,3),np.uint8)
    # dilate
    dilate = cv2.morphologyEx(img,cv2.MORPH_DILATE,kernel)
    # close
    close = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernel)
    # bilateral on both
    blur_close = cv2.bilateralFilter(close,9,75,75)
    blur_dilate = cv2.bilateralFilter(dilate,9,75,75)
    # threshold
    th = cv2.adaptiveThreshold(blur_dilate,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
    # original image
    cv2.imshow('image',img)
    #cv2.imshow('close',close)
    #cv2.imshow('dilate',dilate)
    cv2.imshow('blur_close',blur_close)
    #cv2.imshow('thresh',th)
    cv2.imshow('blur_dilate',blur_dilate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1]:
i = random.randint(0,56)
image_closing('captcha/' + a[i],a[i])

NameError: name 'random' is not defined